Model 2 : RandomForest
- Lags
- date features
- material information
- transport information

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import seaborn as sns
from sklearn.metrics import mean_absolute_error, r2_score
import xgboost as xgb
import gc
import json


In [2]:
Positive_Deliveries = True #Set 0 for negative deliveries
weekend0 = True #Set 0 for positive deliveries during the weekend
Remove_RareOccurences=True
Remove_Outliers=False
AdaptativeFirstDate=False
clear_small_Deliveries=True
forecast_start = pd.Timestamp("2025-01-01")
forecast_end   = pd.Timestamp("2025-05-31")

In [3]:
def print_submission(df_final, filename="submission.csv"):

    df_final["rm_id"] = pd.to_numeric(df_final["rm_id"], errors="coerce").astype("Int64")

    # 3) Load the ID mapping
    ids = pd.read_csv("../data/prediction_mapping.csv")  # or your actual path
    ids["forecast_start_date"] = pd.to_datetime(ids["forecast_start_date"])
    ids["forecast_end_date"]   = pd.to_datetime(ids["forecast_end_date"])
    ids["rm_id"]    = pd.to_numeric(ids["rm_id"], errors="coerce").astype("Int64")
    ids = ids.rename(columns={
        "forecast_end_date": "date"
    })

    # 4) For each ID, get cum at end_date (<= end_date)

    out = (
        ids.merge(df_final, on=["rm_id", "date"], how="left")
        .assign(cum=lambda d: d["cum"].fillna(0))   # cum=0 quand absent
        .sort_values(["rm_id", "date"])
        .reset_index(drop=True)
    )


    # 5) Cum at day before start_date
    submission = (
        out[["ID", "cum"]]
        .rename(columns={"cum": "predicted_weight"})   # if your file needs "predicted_weight"
        .fillna({"predicted_weight": 0})
        .sort_values("ID")
    )

    submission.to_csv("../submissions/" + filename, index=False)
    print(submission.head())


def RemoveOutliers(df,column):
        # Calculate the first quartile (Q1) and third quartile (Q3)
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)

    # Calculate the Interquartile Range (IQR)
    IQR = Q3 - Q1
        # Define the lower and upper bounds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

        # Filter the DataFrame to exclude outliers
    df = df[
        (df[column] >= lower_bound) &
        (df[column] <= upper_bound)
    ]
    return df

def quantile_error(y_pred, y_test, y_test_keys):
    """
    Calculate the average Quantile Loss at the 0.2 level for all materials.

    Args:
        y_pred: Predicted values (1D array, shape [N,]).
        y_test: Actual values (1D array, shape [N,]).
        y_test_keys: DataFrame with columns: rm_id, date (shape [N,]).

    Returns:
        QuantileError_02: Average quantile loss across all materials.
    """
    # Concatenate rm_id, y_test, and y_pred into a single DataFrame
    df = y_test_keys.copy()
    df["y_test"] = y_test
    df["y_pred"] = y_pred

    # Calculate the sum of actual and predicted values for each rm_id
    Ai = df.groupby("rm_id")["y_test"].cumsum()
    Fi = df.groupby("rm_id")["y_pred"].cumsum()

    q=0.2
    # Calculate Quantile Loss at 0.2 level for each rm_id
    quantile_loss = np.maximum(q * (Ai - Fi), (1-q) * (Fi - Ai))

    # Average quantile loss across all materials
    QuantileError_02 = np.mean(quantile_loss)

    return QuantileError_02

def quantile_error_df(df):
    """
    Calculate the average Quantile Loss at the 0.2 level for all materials.

    Args:
        y_pred: Predicted values (1D array, shape [N,]).
        y_test: Actual values (1D array, shape [N,]).
        y_test_keys: DataFrame with columns: rm_id, date (shape [N,]).

        df already contain all these informations
    Returns:
        QuantileError_02: Average quantile loss across all materials.
    """

    # Calculate the sum of actual and predicted values for each rm_id
    Ai = df.groupby("rm_id")["y_test"].cumsum()
    Fi = df.groupby("rm_id")["y_pred"].cumsum()

    q=0.2
    # Calculate Quantile Loss at 0.2 level for each rm_id
    quantile_loss = np.maximum(q * (Ai - Fi), (1-q) * (Fi - Ai))

    # Average quantile loss across all materials
    QuantileError_02 = np.mean(quantile_loss)

    return QuantileError_02

def plot_history_vs_forecast(df_all, df_pred_2025, rm_id,start_date="2023-01-01"):
    # ensure types
    hist = df_all.loc[(df_all["rm_id"] == rm_id) & (df_all["date"] >= pd.Timestamp(start_date)), ["date", "net_weight"]].copy()
    pred = df_pred_2025.loc[df_pred_2025["rm_id"] == rm_id, ["date", "net_weight"]].copy()

    # plot
    plt.figure(figsize=(10, 5)) 
    plt.plot(hist["date"], hist["net_weight"], label="History")
    plt.plot(pred["date"], pred["net_weight"], label="Forecast")

    # reference line for forecast start (optional)
    fs = pd.Timestamp("2025-01-01")
    plt.axvline(fs, linestyle="--")

    plt.title(f"rm_id {rm_id} — net_weight: history vs forecast")
    plt.xlabel("Date")
    plt.ylabel("net_weight")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [4]:
# ---------------------------
# Config: lags & rollings
# ---------------------------
ROLL_WINDOWS    = [7, 14, 30,365]

with open("list_lags.txt") as f:
    ALL_LAGS = json.load(f)

def create_lag_features(data,removeNA=True):
    target = "net_weight"
    d = data

    # --- rolling aggregates (captures trend / momentum) ---
    for window in ROLL_WINDOWS:
        d[f"lf_roll_mean_{window}"] = d.groupby("rm_id")[target].shift(365).rolling(window).mean().reset_index(level=0, drop=True)
        d[f"lf_roll_std_{window}"] = d.groupby("rm_id")[target].shift(365).rolling(window).std().reset_index(level=0, drop=True)

    # Number of deliveries in the same week/month/year 1 year ago
    d["lf_number_deliveries_1y_ago_week"] = d.groupby("rm_id").apply(
        lambda x: (x[target]>0).shift(365).rolling(7).sum(), include_groups=False
    ).reset_index(level=0, drop=True)
    d["lf_number_deliveries_1y_ago_month"] = d.groupby("rm_id").apply(
        lambda x: (x[target]>0).shift(365).rolling(30).sum(), include_groups=False
    ).reset_index(level=0, drop=True)
    d["lf_number_deliveries_1y_ago_year"] = d.groupby("rm_id").apply(
        lambda x: (x[target]>0).shift(365).rolling(365).sum(), include_groups=False
    ).reset_index(level=0, drop=True)

    # --- lag features ---
    lagged_columns = [
        d.groupby("rm_id")[target].shift(lag)
        for lag in ALL_LAGS
    ]
    d = pd.concat([d] + lagged_columns, axis=1)
    d.columns = list(d.columns[:-len(ALL_LAGS)]) + [f"lf_lag_{lag}" for lag in ALL_LAGS]

    # drop first rows with missing lags
    if removeNA:
        d = d.dropna().reset_index(drop=True)

    return d

def create_calendar_features(df,datecol='date'):
    df[f"{datecol}_year"] = df[datecol].dt.year-2004
    df[f"{datecol}_month"] = df[datecol].dt.month
    df[f"{datecol}_week"] = df[datecol].dt.isocalendar().week
    df[f"{datecol}_dayofmonth"] = df[datecol].dt.day 
    df[f"{datecol}_dayofweek"] = df[datecol].dt.dayofweek
    df[f"{datecol}_dayofyear"] = df[datecol].dt.dayofyear
    df[f"{datecol}_isweekend"] = df[datecol].dt.dayofweek.isin([5,6])
    return df





In [5]:
# Delete large objects explicitly
if 'daily_receivals' in globals():
    del daily_receivals
if 'd' in globals():
    del d

# Force garbage collection
gc.collect()

receivals = pd.read_csv('../data/kernel/receivals.csv')
print(len(receivals["rm_id"].unique()))
receivals = receivals.dropna(subset=["net_weight"])
receivals = receivals[receivals["net_weight"] > 0]

receivals['date_arrival'] = (
    pd.to_datetime(receivals['date_arrival'], utc=True)
      .dt.tz_localize(None)
      .dt.normalize()
)

daily_receivals = (
    receivals.groupby(["rm_id","date_arrival"], as_index=False)
             .agg(
                 net_weight_sum=("net_weight", "sum")
             ).rename(columns={"date_arrival": "date","net_weight_sum":"net_weight"})
             .sort_values(["rm_id","date"])
)
daily_receivals["net_weight"] = daily_receivals["net_weight"].astype('float32')

static_fields_pr_rm_id = (
    daily_receivals
    .groupby("rm_id", as_index=False)
    .agg(
        total_net_weight_all_time=("net_weight", "sum"),
        mean_net_weight_all_time=("net_weight", "mean"),
        med_net_weight_all_time=("net_weight", "median"),
        num_deliveries_all_time=("net_weight", lambda x: (x > 0).sum()),
        date_last_delivery=("date", "min")
    )
)
static_fields_pr_rm_id["delay_last_delivery"] = (pd.Timestamp("2025-01-01")- static_fields_pr_rm_id["date_last_delivery"]).dt.days
static_fields_pr_rm_id = static_fields_pr_rm_id.drop(columns=("date_last_delivery"))
if Remove_RareOccurences:
    stats = (
        daily_receivals.groupby("rm_id")
                .agg(
                    last_delivery=("date", "max"),
                    total_deliveries=("date", "count")  # number of delivery-days
                )
    )
    cutoff = pd.Timestamp("2024-12-31") - pd.DateOffset(years=2)
    cond1 = (stats["last_delivery"] <= cutoff) & (stats["total_deliveries"] <= 30 )
    cond2 =(stats["last_delivery"] <= pd.Timestamp("2006-01-01"))
    rare_rm_id = stats.index[cond1 | cond2]
    print(f"We don't have enough information to forecast the following rm_id :\n{rare_rm_id}")
    print(f"number :  {len(rare_rm_id)}")
    
    # I remove rare_rm_id 
    daily_receivals = daily_receivals[~daily_receivals["rm_id"].isin(rare_rm_id)].reset_index(drop=True)



if Remove_Outliers:
    daily_receivals=RemoveOutliers(daily_receivals,"net_weight")

print(f"number of deliveries {len(daily_receivals)}")

full_start = pd.Timestamp("2004-01-01")
full_end   = pd.Timestamp("2024-12-31")
rm_ids = pd.Index(daily_receivals["rm_id"].unique(), name="rm_id")
calendar = pd.date_range(full_start, full_end, freq="D", name="date")
full_idx = pd.MultiIndex.from_product([rm_ids, calendar], names=["rm_id","date"])

daily_receivals = (
    daily_receivals.set_index(["rm_id","date"])["net_weight"]
         .reindex(full_idx, fill_value=0)
         .reset_index()
)



if AdaptativeFirstDate:
    # first actual delivery per rm_id
    first_delivery = (
        daily_receivals
        .loc[daily_receivals["net_weight"] > 0]
        .groupby("rm_id", as_index=False)
        .agg(first_date=("date", "min"))
    )
    
    first_delivery["first_date"] = pd.to_datetime(first_delivery["first_date"])

    start_1st = first_delivery["first_date"] - pd.DateOffset(years=1, months=1)
    start_3y  = forecast_start - pd.DateOffset(years=4)

    # pick the earlier of the two (vectorized, pandas-safe)
    first_delivery["start_date"] = start_1st.where(start_1st <= start_3y, start_3y)

    daily_receivals = (
    daily_receivals
    .merge(first_delivery[["rm_id", "start_date"]], on="rm_id", how="inner")
    .loc[lambda d: d["date"] >= d["start_date"]]
    .drop(columns="start_date")
    .sort_values(["rm_id", "date"])
    .reset_index(drop=True)
)
    

# Static features
# Calculate the total sum of net_weight for each rm_id
# Calculate both the total sum and the count of deliveries > 0



# Merge both back into the original DataFrame
daily_receivals = daily_receivals.merge(
    static_fields_pr_rm_id,
    on="rm_id",
    how="left"
)

daily_receivals=create_lag_features(daily_receivals)
daily_receivals=create_calendar_features(daily_receivals,datecol='date')
daily_receivals = daily_receivals.sort_values(["rm_id","date"]).reset_index(drop=True)
#print(daily_receivals.columns)
print(f"number of rows {len(daily_receivals)}")


204
We don't have enough information to forecast the following rm_id :
Index([ 342.0,  343.0,  345.0,  346.0,  347.0,  348.0,  353.0,  354.0,  355.0,
        357.0,  358.0,  360.0,  362.0,  364.0,  365.0,  366.0,  367.0,  368.0,
        369.0,  374.0,  375.0,  378.0,  379.0,  380.0,  381.0,  383.0,  387.0,
        388.0,  389.0,  390.0, 1842.0, 1843.0, 1844.0, 1845.0, 1846.0, 1850.0,
       1851.0, 1852.0, 1853.0, 1854.0, 1857.0, 1858.0, 1866.0, 1867.0, 1868.0,
       1871.0, 1872.0, 1873.0, 1874.0, 1882.0, 1981.0, 2001.0, 2061.0, 2102.0,
       2121.0, 2122.0, 2128.0, 2139.0, 2141.0, 2148.0, 2156.0, 2158.0, 2201.0,
       2223.0, 2282.0, 2283.0, 2285.0, 2302.0, 2304.0, 2322.0, 2323.0, 2341.0,
       2343.0, 2344.0, 2345.0, 2347.0, 2348.0, 2362.0, 2363.0, 2402.0, 2421.0,
       2481.0, 2521.0, 2561.0, 2742.0, 2821.0, 2841.0, 2861.0, 3022.0, 3101.0,
       3144.0, 3222.0, 3241.0, 3481.0, 3501.0, 3541.0],
      dtype='float64', name='rm_id')
number :  96
number of deliveries 40321
number

In [28]:
def flag_any(x, patterns):
    if not isinstance(x, str):
        return 0
    x = x.lower()
    return int(any(p in x for p in patterns))

In [29]:
materials = pd.read_csv('../data/extended/materials.csv')
# I don't manage the deleted localisation -> next feature ?
print(len(materials))
print(materials.columns)
materials = materials.dropna(subset=["rm_id"])

materials["fam_6xxx"] = materials["raw_material_alloy"].map(lambda x: flag_any(x, ["6060","6063","6082","6005","6016"]))
materials["fam_5xxx"] = materials["raw_material_alloy"].map(lambda x: flag_any(x, ["5000","5754"]))
materials["fam_1xxx"] = materials["raw_material_alloy"].map(lambda x: flag_any(x, ["1050","1100"]))
materials["fam_pure"] = materials["raw_material_alloy"].map(lambda x: flag_any(x, ["99.5","99,5","99.7","99,7","cspa","pm al","granalla","coil"]))
materials["fam_scrap_shred"] = materials["raw_material_alloy"].map(lambda x: flag_any(x, ["shredd","tritura","scrap"]))
materials["fam_briquetas"] = materials["raw_material_alloy"].map(lambda x: flag_any(x, ["briquet","compact"]))
materials["fam_mele"] = materials["raw_material_alloy"].map(lambda x: flag_any(x, ["melé","cacharro"]))
materials["fam_master"] = materials["raw_material_alloy"].map(lambda x: flag_any(x, [" si "," mg "," mn "," cu "," cr "," fe ","ti-bors","wire","cable"]))
materials["treat_lacado"] = materials["raw_material_alloy"].map(lambda x: flag_any(x, ["lac","paint"]))
materials["treat_anodizado"] = materials["raw_material_alloy"].map(lambda x: flag_any(x, ["anod"]))
materials["treat_thermal_break"] = materials["raw_material_alloy"].map(lambda x: flag_any(x, ["thermal break","puente t"]))

materials["loc_SB"] = materials["stock_location"].map(lambda x: flag_any(x, ["sb"]))
materials["loc_SA"] = materials["stock_location"].map(lambda x: flag_any(x, ["sa"]))
materials["loc_SC"] = materials["stock_location"].map(lambda x: flag_any(x, ["sc"]))
materials["loc_AA"] = materials["stock_location"].map(lambda x: flag_any(x, ["aa"]))
materials["loc_PM"] = materials["stock_location"].map(lambda x: flag_any(x, ["pm "]))

materials["loc_ThermalBreak"] = materials["stock_location"].map(lambda x: flag_any(x, ["thermal break","puente t"," pt"]))
materials["loc_Lavadero"] = materials["stock_location"].map(lambda x: flag_any(x, ["lavadero"]))
materials["loc_Granalla"] = materials["stock_location"].map(lambda x: flag_any(x, ["granalla"]))
materials["loc_Briquetas"] = materials["stock_location"].map(lambda x: flag_any(x, ["briquet"]))


# your flag columns already created above (fam_*, treat_*, loc_*)
flag_cols = [c for c in materials.columns if c.startswith(("fam_", "treat_", "loc_"))]

# Group and aggregate:
# - flags: max (so any 1 across the rm_id becomes 1)
# - format type: mean
# - keep a representative text for context (first)

materials_by_rm = (
    materials
    .groupby("rm_id", as_index=False)
    .agg({**{c: "mean" for c in flag_cols},
            "raw_material_format_type": "mean"})
)

# (optional) if you prefer ints for flags (0/1), cast them:
for c in flag_cols:
    materials_by_rm[c] = materials_by_rm[c].astype(int)


1218
Index(['rm_id', 'product_id', 'product_version', 'raw_material_alloy',
       'raw_material_format_type', 'stock_location'],
      dtype='object')


In [30]:
transportation = pd.read_csv('../data/extended/transportation.csv')

transportation = transportation.dropna(subset=["purchase_order_id","net_weight","rm_id"])

#df_nan = transportation[transportation["purchase_order_id"].isna()] # no more nan values
#print(df_nan) #now empty
weight_cols = ["vehicle_start_weight","vehicle_end_weight",
               "gross_weight","tare_weight","net_weight"]
for c in weight_cols:
    transportation[c] = pd.to_numeric(transportation[c], errors="coerce")

# --- One-hot encode transporter_name -> 0/1 columns
transp_dum = pd.get_dummies(
    transportation["transporter_name"],
    prefix="_",
    dummy_na=False  # set True if you want a "transp_nan" column for missing
)

# attach dummies
transportation = pd.concat([transportation, transp_dum], axis=1)

# --- Build aggregation dict:
#     - transporter dummies: mean (→ proportion of trips with that transporter per rm_id)
#     - weights: mean
agg_dict = {**{c: "max" for c in transp_dum.columns},
            **{c: "mean" for c in weight_cols}}

transportation_by_rm = (
    transportation
    .groupby("rm_id", as_index=False)
    .agg(agg_dict)
)

# pretty rounding for proportions/means
transportation_by_rm[transp_dum.columns] = transportation_by_rm[transp_dum.columns].round(3)
transportation_by_rm[weight_cols] = transportation_by_rm[weight_cols].round(3)
transportation_by_rm=transportation_by_rm.rename(columns={'net_weight': 'transp_mean_weight'})
print(transportation_by_rm.columns)

Index(['rm_id', '__Transporter0', '__Transporter1', '__Transporter10',
       '__Transporter11', '__Transporter12', '__Transporter13',
       '__Transporter14', '__Transporter15', '__Transporter16',
       '__Transporter17', '__Transporter18', '__Transporter19',
       '__Transporter2', '__Transporter20', '__Transporter21',
       '__Transporter22', '__Transporter23', '__Transporter24',
       '__Transporter3', '__Transporter4', '__Transporter5', '__Transporter6',
       '__Transporter7', '__Transporter8', '__Transporter9',
       'vehicle_start_weight', 'vehicle_end_weight', 'gross_weight',
       'tare_weight', 'transp_mean_weight'],
      dtype='object')


In [31]:
df_all = (
    daily_receivals
    .merge(materials_by_rm, on="rm_id", how="left")
    .merge(transportation_by_rm, on="rm_id", how="left")
)
df_all = df_all.sort_values(["rm_id","date"]).reset_index(drop=True)

if False: # you can change the value if you train the model again without the same columns

    worst_features = [
        "__Transporter0", "__Transporter1", "__Transporter2", "__Transporter3", "__Transporter4",
        "__Transporter5", "__Transporter6", "__Transporter7", "__Transporter8", "__Transporter9",
        "__Transporter10", "__Transporter11", "__Transporter12", "__Transporter13", "__Transporter14",
        "__Transporter15", "__Transporter16", "__Transporter17", "__Transporter18", "__Transporter19",
        "__Transporter20", "__Transporter21", "__Transporter23", "__Transporter24",
        "loc_AA", "loc_PM", "loc_SC", "loc_Briquetas", "loc_Granalla", "loc_ThermalBreak",
        "fam_1xxx", "fam_5xxx", "fam_briquetas", "fam_mele", "fam_master",
        "treat_thermal_break", "treat_lacado"
    ]
    # => worst_features contains features with the smallest importance after a training.
    df_all=df_all.drop(columns=worst_features)


print(df_all.columns)
print(len(df_all.columns))


Index(['rm_id', 'date', 'net_weight', 'total_net_weight_all_time',
       'num_deliveries_all_time', 'lf_roll_mean_7', 'lf_roll_std_7',
       'lf_roll_mean_14', 'lf_roll_std_14', 'lf_roll_mean_30',
       ...
       '__Transporter5', '__Transporter6', '__Transporter7', '__Transporter8',
       '__Transporter9', 'vehicle_start_weight', 'vehicle_end_weight',
       'gross_weight', 'tare_weight', 'transp_mean_weight'],
      dtype='object', length=296)
296


In [32]:
# Optionnal create the file to see what is my input data
#df_all.to_csv("df_all")

In [33]:
columns_keys=["rm_id","date"]
columns_output = ['net_weight']
columns_input = [c for c in df_all.columns if c not in ["rm_id","date","net_weight"]]

def split_set_Train_Test(df_all, test_size=0.05):
    if test_size == 2024:
        train = df_all.groupby('rm_id').apply(
            lambda x: x.sort_values('date').iloc[:-366])
        test = df_all.groupby('rm_id').apply(
            lambda x: x.sort_values('date').iloc[-366:])
    elif test_size == 0.:
        train = df_all.groupby('rm_id').apply(
            lambda x: x.sort_values('date').iloc[:-1])
        test = df_all.groupby('rm_id').apply(
            lambda x: x.sort_values('date').iloc[-1:])

    else:
        train = df_all.groupby('rm_id').apply(
            lambda x: x.sort_values('date').iloc[:int((1-test_size) * len(x))])
        test = df_all.groupby('rm_id').apply(
            lambda x: x.sort_values('date').iloc[int((1-test_size) * len(x)):])

    X_train_keys= train[columns_keys]
    X_train = train[columns_input]
    y_train = train[columns_output].values.ravel()

    X_test_keys= test[columns_keys]
    X_test = test[columns_input]
    y_test = test[columns_output].values.ravel()

    X_train_keys=X_train_keys.reset_index(drop=True)
    X_test_keys=X_test_keys.reset_index(drop=True)
    
    return X_train_keys, X_train, y_train, X_test_keys, X_test, y_test

In [34]:
X_train_keys,X_train, y_train, X_test_keys, X_test, y_test = split_set_Train_Test(df_all, test_size=2024)

C:\Users\julie\AppData\Local\Temp\ipykernel_10908\3598047402.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train = df_all.groupby('rm_id').apply(
C:\Users\julie\AppData\Local\Temp\ipykernel_10908\3598047402.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test = df_all.groupby('rm_id').apply(


In [ ]:
#Let's find the best model
# Train XGBoost

mae_loss_min = np.inf
quantile_loss_min=np.inf
Best_parameters_mae={"n_estimators":0,"learning_rate":0,"max_depth":0}
Best_parameters_ql={"n_estimators":0,"learning_rate":0,"max_depth":0}

for n_estimators in [500, 1000, 1500, 2000,3000]:
    for learning_rate in [0.1, 0.05, 0.02]:
        for max_depth in [4, 6, 8, 10]:
            print(f"Training: n_estimators={n_estimators}, lr={learning_rate}, depth={max_depth}")
            
            model = xgb.XGBRegressor(
                objective="reg:squarederror",
                n_estimators=n_estimators,
                learning_rate=learning_rate,
                max_depth=max_depth,
                subsample=0.8,
                colsample_bytree=0.8,
                random_state=42,
                n_jobs=-1,
                tree_method="hist"
            )
            
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            mae_error = mean_absolute_error(y_test, y_pred)
            quantile_error_net_weight = quantile_error(y_pred,y_test,X_test_keys)



            if mae_error < mae_loss_min:
                Best_parameters_mae={
                "n_estimators": n_estimators,
                "learning_rate": learning_rate,
                "max_depth": max_depth
                }
                mae_loss_min= mae_error
                print(f"! Better Parameters ! error = {mae_error}")
            else:
                print(f"error = {mae_error}")

            if quantile_error_net_weight < quantile_loss_min:
                Best_parameters_ql={
                "n_estimators": n_estimators,
                "learning_rate": learning_rate,
                "max_depth": max_depth
                }
                quantile_loss_min=quantile_error_net_weight


print(f"The best parameters for mae are {Best_parameters_mae}")
print(f"The best parameters for qf are {Best_parameters_ql}")


Training: n_estimators=500, lr=0.1, depth=4
! Better Parameters ! error = 2212.75732421875
Training: n_estimators=500, lr=0.1, depth=6
error = 2268.29541015625
Training: n_estimators=500, lr=0.1, depth=8
error = 2400.0751953125
Training: n_estimators=500, lr=0.1, depth=10
error = 2435.28076171875
Training: n_estimators=500, lr=0.05, depth=4
! Better Parameters ! error = 2172.006103515625
Training: n_estimators=500, lr=0.05, depth=6
error = 2200.416748046875
Training: n_estimators=500, lr=0.05, depth=8
error = 2206.4208984375
Training: n_estimators=500, lr=0.05, depth=10
error = 2339.19921875
Training: n_estimators=500, lr=0.02, depth=4
! Better Parameters ! error = 2161.25634765625
Training: n_estimators=500, lr=0.02, depth=6
! Better Parameters ! error = 2141.913330078125
Training: n_estimators=500, lr=0.02, depth=8
error = 2156.528564453125
Training: n_estimators=500, lr=0.02, depth=10
error = 2175.6083984375
Training: n_estimators=1000, lr=0.1, depth=4
error = 2234.895751953125
Trai

KeyboardInterrupt: 